In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import os
from transformers import BertTokenizer, AutoModelForSequenceClassification,BertModel
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


class TextDataset(Dataset):
    def __init__(self, tokenizer, directory, max_len=512):
        self.tokenizer = tokenizer
        self.texts = []
        self.labels = []

        # Example: Read each file and assign a label based on the filename or content
        for filename in os.listdir(directory):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                text = file.read().strip()
                self.texts.append(text)
                # Assuming labels are somehow derived from filenames or content
                self.labels.append(int(filename[0]))  # Simplistic example

        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, max_length=self.max_len, padding='max_length', truncation=True)

        return {
            'input_ids': torch.tensor(inputs['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(inputs['attention_mask'], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = BertModel.from_pretrained('bert-base-cased', output_attentions=True)
# Create dataset
dataset = TextDataset(tokenizer, '/home/angel/universe/kv_version/dataset/')

# You can now pass 'dataset' to the Trainer


In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    compute_metrics=compute_metrics,
)

# Specify the path where you want to save the model
save_directory = "/home/angel/universe/test_trainer"

# Save the model
trainer.save_model(save_directory)

# Optionally save the tokenizer associated with your model
tokenizer.save_pretrained(save_directory)


('/home/angel/universe/test_trainer/tokenizer_config.json',
 '/home/angel/universe/test_trainer/special_tokens_map.json',
 '/home/angel/universe/test_trainer/vocab.txt',
 '/home/angel/universe/test_trainer/added_tokens.json',
 '/home/angel/universe/test_trainer/tokenizer.json')

In [7]:
import torch
from transformers import BertTokenizer, AutoModelForSequenceClassification
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure the save_directory is defined correctly
save_directory = "/home/angel/universe/test_trainer"

# Load the fine-tuned model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(save_directory, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(save_directory)

max_length = 512
chunk_size = 400  # A bit less than max_length to allow for special tokens
stride = 50  # Overlapping tokens to avoid loss of context

# Read the file
file_path = "/home/angel/universe/kv_version/dataset/1_llm.txt"
with open(file_path, 'r') as file:
    input_text = file.read()

# Tokenize the input text with chunking for long sequences
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_length)

# Get model outputs, including attentions
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(**inputs)
    attentions = outputs.attentions  # Tuple of attention tensors for each layer

# Choose a specific layer and head to visualize
layer_index = -5  # Middle layer
head_index = 0  # First head

# Extract the attention matrix for the specified layer and head
attention = attentions[layer_index][0, head_index].detach().cpu().numpy()
attention = attention[:,1:]

# Convert token IDs back to tokens for labeling
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])



In [8]:


# Filter and sort the attention scores above 0.5
# Create a mask where attention values are greater than 0.5
high_attention_mask = attention > 0.01

# Apply mask to the attention matrix to filter values
high_attention_values = attention[high_attention_mask]

# Get the corresponding pairs of tokens for each high attention value
token_pairs = [(tokens[i], tokens[j]) for i, j in zip(*np.where(high_attention_mask))]

# Sort the pairs by attention values in descending order
sorted_pairs = sorted(zip(token_pairs, high_attention_values), key=lambda x: x[1], reverse=True)

import re

# Define a regex pattern that matches undesired tokens containing special characters
pattern = re.compile(r'[#.\-%$]')

# Filter and sort the attention scores above 0.5, excluding pairs with special characters
# Note: This step assumes 'sorted_pairs' has already been defined using the previous snippet
cleaned_pairs = [
    (pair, score) for pair, score in sorted_pairs 
    if not (pattern.search(pair[0]) or pattern.search(pair[1]))
]

# List of common English prepositions
prepositions = set([
    'aboard', 'about', 'above', 'across', 'after', 'against', 'along', 'amid', 'among',
    'anti', 'around', 'as', 'at', 'before', 'behind', 'below', 'beneath', 'beside', 'besides',
    'between', 'beyond', 'but', 'by', 'concerning', 'considering', 'despite', 'down', 'during',
    'except', 'excepting', 'excluding', 'following', 'for', 'from', 'in', 'inside', 'into',
    'like', 'minus', 'near', 'of', 'off', 'on', 'onto', 'opposite', 'outside', 'over',
    'past', 'per', 'plus', 'regarding', 'round', 'save', 'since', 'than', 'through', 'to',
    'toward', 'towards', 'under', 'underneath', 'unlike', 'until', 'up', 'upon', 'versus',
    'via', 'with', 'within', 'without','the','is','and'
])

# Filter and sort the attention scores above 0.5, excluding pairs with special characters or prepositions
cleaned_pairs = [
    (pair, score) for pair, score in sorted_pairs 
    if not (pattern.search(pair[0]) or pattern.search(pair[1]) or
            pair[0].lower() in prepositions or pair[1].lower() in prepositions or 
            len(pair[0]) == 1 or len(pair[1]) == 1)  # Check if either token is a single character)
]

# Display the cleaned, high attention token pairs with their scores
for (token1, token2), score in cleaned_pairs:
    print(f"Attention between '{token1}' and '{token2}': {score:.2f}")

import pandas as pd

# Convert cleaned_pairs to a DataFrame
data = [(pair[0], pair[1], score) for (pair, score) in cleaned_pairs]
df = pd.DataFrame(data, columns=["Token1", "Token2", "Score"])

df


Attention between 'existing' and 'We': 0.99
Attention between 'Han' and 'We': 0.99
Attention between 'Fe' and 'We': 0.99
Attention between 'method' and 'We': 0.98
Attention between 'Self' and 'We': 0.98
Attention between 'motivation' and 'We': 0.97
Attention between 'studies' and 'We': 0.97
Attention between 'LL' and 'We': 0.97
Attention between 'LL' and 'We': 0.97
Attention between 'hall' and 'We': 0.97
Attention between 'sampling' and 'We': 0.97
Attention between 'reasoning' and 'We': 0.97
Attention between 'explanations' and 'We': 0.95
Attention between 'modules' and 'We': 0.95
Attention between 'Internal' and 'We': 0.94
Attention between 'reasoning' and 'We': 0.94
Attention between 'yet' and 'stream': 0.94
Attention between 'phenomena' and 'We': 0.93
Attention between 'stream' and 'We': 0.93
Attention between 'theoretical' and 'We': 0.93
Attention between 'or' and 'We': 0.93
Attention between 'accurately' and 'We': 0.93
Attention between 'two' and 'We': 0.93
Attention between 'Qing

,Token1,Token2,Score
0,existing,We,0.989953
1,Han,We,0.989828
2,Fe,We,0.989488
3,method,We,0.982627
4,Self,We,0.975270
...,...,...,...
1166,propose,Almost,0.010060
1167,or,itself,0.010050
1168,Ex,Furthermore,0.010032
1169,enhance,del,0.010020


In [ ]:
#convert the dataframe to json file 
json_data = df.to_json(orient="records", lines=True)

# Save JSON to a file
output_path = 'nvda_pairs.json'
with open(output_path, 'w') as file:
    file.write(json_data)